Percobaan ketiga dengan transfer learning CNN menggunakan model DenseNet121 </br>
Training dilakukan dengan 100 epochs </br>
Model yang dihasilkan punya akurasi validasi (test) {TBD}. Nilai tersebut diperoleh pada epoch ke {TBD} </br>
History dari training disimpan di file historyModel3 </br>

<h2> ====IMPORT LIBRARY==== </h2>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow
from tensorflow.keras import optimizers, regularizers, Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout,SpatialDropout2D, BatchNormalization
from tensorflow.keras.models  import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.applications import densenet
import numpy as np
import os
import time
import glob
import pickle
import matplotlib.pyplot as plt

D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\tubes-tf\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

<h2>====DATASET INPUT====</h2>

In [2]:
path = "dataset-resized"
listImage = glob.glob(os.path.join(path, '*/*.jpg'))
print("Jumlah dataset: " + str(len(listImage)))

Jumlah dataset: 2527


<h2> ====IMAGE AUGMENTATION==== </h2>

Membuat image augmentation pada dataset yang sudah ada untuk memperbanyak jumlah dataset</br>
Membagi dataset yang ada menjadi dataset untuk training dan test </br>



In [3]:
train = ImageDataGenerator(horizontal_flip=True,
                          validation_split=0.1,
                          rescale=1./255,
                          shear_range=0.2,
                          zoom_range=0.2,
                          width_shift_range=0.2,
                          height_shift_range=0.2)

test = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = train.flow_from_directory(path, 
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode="categorical",
                                            subset="training")

test_generator = test.flow_from_directory(path,
                                          target_size=(224,224),
                                          batch_size=32,
                                          class_mode="categorical",
                                          subset="validation")

#label untuk image
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.
{0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}


In [4]:
for img,label in train_generator:
    break
img.shape, label.shape

((32, 224, 224, 3), (32, 6))

In [5]:
Labels = '\n'.join(sorted(train_generator.class_indices.keys()))

<h2> ====CNN (Convolutional Neural Network)==== </h2>

Load model dari DenseNet121

In [6]:
base_model = densenet.DenseNet121(include_top=False,
                                  weights="imagenet",
                                  input_tensor=None,
                                  input_shape=(224,224,3),
                                  pooling="max")

x = base_model.output
x = Dense(256, activation="elu", kernel_regularizer=regularizers.l2(0.1))(x)
x = Dropout(0.6)(x)
x = BatchNormalization()(x)
predictions = Dense(6, activation="softmax")(x)

model = Model(base_model.input, outputs=predictions)

In [7]:
#menyimpan model
modelFile = "modelConfigTransferLearning.h5"
checkpoint1 = ModelCheckpoint(modelFile, monitor="val_acc", verbose=1, save_best_only=True, mode="max")
callback_list = [checkpoint1]

In [8]:
optimizer = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

<h2>====TRAINING====</h2>

In [9]:
%%time
history = model.fit_generator(train_generator,
                              epochs=70,
                              steps_per_epoch=len(train_generator),
                              validation_data=test_generator,
                              validation_steps=len(test_generator))

Epoch 1/70


ResourceExhaustedError: OOM when allocating tensor with shape[32,192,28,28] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/SGD/gradients/zeros_616 = Fill[T=DT_FLOAT, _class=["loc:@training/SGD/gradients/AddN_317"], index_type=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/SGD/gradients/Shape_209, training/SGD/gradients/zeros_616/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training/SGD/gradients/pool2_bn/cond/FusedBatchNorm_grad/FusedBatchNormGrad/_8051 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_24248...chNormGrad", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a 

In [ ]:
#save history ke file
with open('historyModelTransferLearning', 'wb') as f :
    pickle.dump(history.history, f)

In [ ]:
train_acc = history.history['acc']
validation_acc = history.history['val_acc']

train_loss = history.history['loss']
validation_loss = history.history['val_loss']

#plot grafik akurasi
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(train_acc, label='Training Accuracy')
plt.plot(validation_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

#plot grafik loss
plt.subplot(2, 1, 2)
plt.plot(train_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()